In [98]:
import nltk
import urllib
import pandas as pd
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from textblob import TextBlob
from nltk.util import ngrams
from collections import Counter
from operator import itemgetter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse.linalg import svds
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import os
import re
from sklearn.manifold import TSNE
from collections import defaultdict
from nltk.corpus import stopwords
import gensim
from sklearn.manifold import TSNE
import gensim
import string
from gensim import corpora

In [5]:
pride = urllib.request.urlopen('https://www.gutenberg.org/files/1342/1342-0.txt').read()
pride_lines = pride.decode('utf8')

In [111]:
def find_chapters(book):
    chapters = {}
    i = 0
    b = 0
    for x in range(1, 62):
        i = book.find("Chapter", + b)
        b = book.find("Chapter", i + 1)
        chapters['Chapter {}'.format(x)] = book[i:b]
    return chapters

In [143]:
stop = stopwords.words('english') + ['mr',
                                     'mrs',
                                     'miss', 
                                     'say',
                                     'have', 
                                     'might',
                                     'thought',
                                     'would', 
                                     'could', 
                                     'make', 
                                     'much',
                                     'dear',
                                     'must',
                                     'know',
                                     'one',
                                     'good',
                                     'every',
                                     'towards',
                                     'give']

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'
    
def lemmatize_word(word):
    lemmatizer = WordNetLemmatizer()
    try:
        tag = get_wordnet_pos(nltk.pos_tag([word])[0][1])
        return lemmatizer.lemmatize(word, pos=tag)
    except:
        pass

In [121]:
def all_sentences(Object):
    vocab = []
    for line in sent_tokenize(Object):
        line= re.sub('[%s]|Chapter' % re.escape(string.punctuation), '', line)
        line = re.sub('[^a-zA-Z\ ]', '', line)
        line = line.lower()
        line = line.split()
        line = [lemmatize_word(x) for x in line if lemmatize_word(x) not in stop]
        vocab.append(line)
        
    return vocab

In [146]:
def get_frequecies(corpus):
    dictionary = corpora.Dictionary(corpus)
    # You can check the mapping by caling 'token2id' attribute.
    id_map = dictionary.token2id
    count_map = dictionary.dfs
    word_count = {}
    for word, id_ in id_map.items():
        word_count[word] = count_map[id_]
    sorted_corpus = sorted([(x,y) 
                           for x,y in word_count.items()], 
                           key=lambda x: x[1], reverse=True)
    return sorted_corpus

### Top Keywords in Full Corpus

In [147]:
full_corpus = all_sentences(pride_lines[2229:])
frequencies = get_frequecies(full_corpus)

In [148]:
frequencies[:20]

[('elizabeth', 613),
 ('darcy', 350),
 ('bennet', 314),
 ('well', 287),
 ('sister', 286),
 ('see', 278),
 ('go', 267),
 ('lady', 252),
 ('bingley', 250),
 ('jane', 245),
 ('come', 243),
 ('think', 227),
 ('time', 217),
 ('though', 215),
 ('never', 212),
 ('soon', 208),
 ('may', 192),
 ('little', 182),
 ('take', 182),
 ('great', 179)]

### Total Unique Words

In [149]:
len(frequencies)

5396

### Top Keywords in Each Chapter

In [145]:
text = find_chapters(pride_lines[2229:])
for chapter in ['Chapter {}'.format(x) for x in range(1, 62)]:
    corpus = all_sentences(text[chapter])
    print('{}'.format(chapter), get_frequecies(corpus, max_words=5))

Chapter 1 [('bennet', 5), ('see', 5), ('visit', 5), ('man', 4), ('single', 4)]
Chapter 2 [('bennet', 8), ('bingley', 5), ('visit', 4), ('acquaintance', 4), ('though', 3)]
Chapter 3 [('bingley', 15), ('dance', 14), ('bennet', 11), ('lady', 8), ('room', 8)]
Chapter 4 [('bingley', 11), ('never', 6), ('like', 6), ('darcy', 6), ('sister', 5)]
Chapter 5 [('lucas', 7), ('bennet', 6), ('think', 4), ('well', 4), ('like', 4)]
Chapter 6 [('darcy', 12), ('though', 10), ('elizabeth', 10), ('dance', 10), ('well', 9)]
Chapter 7 [('elizabeth', 14), ('bennet', 11), ('well', 11), ('go', 10), ('jane', 10)]
Chapter 8 [('bingley', 20), ('sister', 14), ('elizabeth', 13), ('darcy', 10), ('lady', 7)]
Chapter 9 [('bingley', 14), ('bennet', 9), ('country', 9), ('elizabeth', 8), ('mother', 8)]
Chapter 10 [('bingley', 17), ('darcy', 16), ('elizabeth', 15), ('letter', 8), ('write', 8)]
Chapter 11 [('darcy', 12), ('elizabeth', 11), ('bingley', 8), ('laugh', 6), ('hurst', 6)]
Chapter 12 [('elizabeth', 6), ('jane', 6